In [ ]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML
import os
import pandas as pd
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html


%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from utils.utils import *

In [ ]:
import networkx as nx
from networkx.readwrite import json_graph
import pickle

def load_sql_rep(fn, dummy=None):
    assert ".pkl" in fn
    try:
        with open(fn, "rb") as f:
            query = pickle.load(f)
    except Exception as e:
        print(e)
        print(fn + " failed to load...")
        exit(-1)

    query["subset_graph"] = \
            nx.OrderedDiGraph(json_graph.adjacency_graph(query["subset_graph"]))
    query["join_graph"] = json_graph.adjacency_graph(query["join_graph"])
    if "subset_graph_paths" in query:
        query["subset_graph_paths"] = \
                nx.OrderedDiGraph(json_graph.adjacency_graph(query["subset_graph_paths"]))

    return query

In [ ]:

#QDIR = "./queries/tpcds/all"
#QDIR = "queries/job/all_job/sqls/all_job"
#QDIR = "ceb-unique-sqls"

#WK = "tpcds1"
#WK = "tpch1"
#WK = "tpcds"
#WK = "job"
WK = "ceb"

#WK = "tpch"
#WK = "stack"

WORKLOADS = {}
WORKLOADS["stack"] = "../scraping-queries/sqls"
WORKLOADS["tpcds"] = "../data/tpcds/all/"
WORKLOADS["tpcds1"] = "../data/tpcds1/all/"
WORKLOADS["tpch"] = "../data/tpch/all/"
WORKLOADS["tpch1"] = "../data/tpch1/all/"
WORKLOADS["job"] = "../data/job/all_job/sqls"
WORKLOADS["ceb"] = "../data/ceb-all/sqls"

QDIR = WORKLOADS[WK]
OUTPATH = os.path.join(QDIR, "dfs")

make_dir(OUTPATH)

#QDIR = "../scraping-queries/sqls"
#QDIR = "../data/job/all_job/sqls"
#QDIR = "../data/tpcds/all/"

COL_TABLES_FN = {}
COL_TABLES_FN["tpcds"] = "../data/tpcds/tpcds.sql"
COL_TABLES_FN["tpcds1"] = "../data/tpcds/tpcds.sql"
COL_TABLES_FN["tpch"] = "../data/tpch/tpch.sql"
COL_TABLES_FN["tpch1"] = "../data/tpch/tpch.sql"

#TPCDS_TABLES=
COLS_TO_TABLES = {}


In [361]:
if "tpc" in QDIR:
    CTABLES = COL_TABLES_FN[WK]
    with open(CTABLES) as f:
        tdata = f.readlines()

    for li, line in enumerate(tdata):
        #print(line)
        line = line.lower()
        if "create table" in line:
            #print(line)
            vals = line.split(" ")
            #print("table: ", vals[2])
            table = vals[2].replace("\n", "")

            for i in range(1000):
                cline = tdata[li+i]
                cline = cline.lower()
                cline = cline.lstrip()
                print(cline)

                if "create table" in cline or "primary key" in cline:
                    continue
                vals = cline.split(" ")
                col = vals[0]

                if not (")" in col or "(" in col or col == ""):
                    if col in COLS_TO_TABLES:
                        print("?>>???")
                        print(col)
                    assert col not in COLS_TO_TABLES   
                    COLS_TO_TABLES[col] = table

                if ")" in cline and "," not in cline:
                    break
        

In [362]:
print(COLS_TO_TABLES)

{}


In [363]:
for k,v in COLS_TO_TABLES.items():
    if "tpch.dbo" in v:
        COLS_TO_TABLES[k] = v.replace("tpch.dbo.", "")

In [364]:
qdict = {}
qfns = []
QREP_DIR = QDIR.replace("sqls", "")
if WK in ["job", "ceb"]:
    qfns = os.listdir(QREP_DIR)
    for fn in qfns:
        qfn = os.path.join(QREP_DIR, fn)
        if ".pkl" in qfn:
            qdict[fn] = load_sql_rep(qfn)

/tmp/ipykernel_285756/1110685569.py:16: DeprecationWarning: OrderedDiGraph is deprecated and will be removed in version 3.0.
Use `DiGraph` instead, which guarantees order is preserved for
Python >= 3.7

  nx.OrderedDiGraph(json_graph.adjacency_graph(query["subset_graph"]))


In [365]:
FNS = os.listdir(QDIR)

In [366]:
sqls = []
qreps = []

for fi, fn in enumerate(FNS):
    if ".sql" not in fn:
        continue
    if "all" in fn:
        continue
    
    fnkey = fn.replace(".sql", ".pkl")
    qreps.append(qdict[fnkey])
        
    fn = os.path.join(QDIR, fn)
    with open(fn, "r") as f:
        sql = f.read()

    sql = sql.replace("IN", "in")
    sql = sql.replace("In", "in")
    sql = sql.replace("ILIKE", "like")
    sql = sql.replace("LIKE", "like")
    
    #sql = sql.replace("ILIKE", "like")
    #sql = sql.replace("LIKE", "like")
    if "return_rank" in sql:
        print(sql)
    sqls.append(sql)

In [367]:
print(len(sqls), len(qreps))

3133 3133


In [368]:
for i, sql in enumerate(sqls):
    sql = sql.replace("IN", "in")
    sql = sql.replace("In", "in")
    sql = sql.replace("ILIKE", "like")
    sql = sql.replace("LIKE", "like")
    sqls[i] = sql

In [369]:
len(sqls)

3133

In [370]:
sql = sqls[18]

In [371]:
print(sql)

SELECT n.gender, rt.role, cn.name, COUNT(*)
FROM title as t,
movie_companies as mc,
company_name as cn,
company_type as ct,
kind_type as kt,
cast_info as ci,
name as n,
role_type as rt,
movie_info as mi1,
info_type as it1,
person_info as pi,
info_type as it2
WHERE t.id = mc.movie_id
AND t.id = ci.movie_id
AND t.id = mi1.movie_id
AND mi1.movie_id = ci.movie_id
AND ci.movie_id = mc.movie_id
AND cn.id = mc.company_id
AND ct.id = mc.company_type_id
AND kt.id = t.kind_id
AND ci.person_id = n.id
AND ci.role_id = rt.id
AND mi1.info_type_id = it1.id
AND n.id = pi.person_id
AND pi.info_type_id = it2.id
AND ci.person_id = pi.person_id
AND (kt.kind in ('episode','movie','tv mini series','tv movie','tv series','video game','video movie'))
AND (rt.role in ('director','guest','production designer'))
AND (t.production_year <= 1990)
AND (t.production_year >= 1945)
AND (it1.id in ('8'))
AND (mi1.info like '%v%')
AND (pi.info like '%aug%')
AND (it2.id in ('21'))
GROUP BY n.gender, rt.role, cn.name
ORDER

In [372]:
#opdf,exprdf = parse_sqls_single([sql], 0, COLS_TO_TABLES)
#opdf
parse_sql_preds(sql)

,op,dtype,optype,constant,input,column,opstring,is_id
0,=,string,in,'episode',kt,kind,"kt.kind in ('episode','movie','tv mini series'...",0
1,=,string,in,'movie',kt,kind,"kt.kind in ('episode','movie','tv mini series'...",0
2,=,string,in,'tv mini series',kt,kind,"kt.kind in ('episode','movie','tv mini series'...",0
3,=,string,in,'tv movie',kt,kind,"kt.kind in ('episode','movie','tv mini series'...",0
4,=,string,in,'tv series',kt,kind,"kt.kind in ('episode','movie','tv mini series'...",0
5,=,string,in,'video game',kt,kind,"kt.kind in ('episode','movie','tv mini series'...",0
6,=,string,in,'video movie',kt,kind,"kt.kind in ('episode','movie','tv mini series'...",0
7,=,string,in,'director',rt,role,"rt.role in ('director','guest','production des...",0
8,=,string,in,'guest',rt,role,"rt.role in ('director','guest','production des...",0
9,=,string,in,'production designer',rt,role,"rt.role in ('director','guest','production des...",0


In [376]:
start = time.time()
opdf,exprdf = parse_sqls_par(sqls, COLS_TO_TABLES)
print("parse sqls took: ", int(time.time()-start))
print("Num Ops: ", len(opdf), "Num Expressions: ", len(exprdf))

number of sql groups:  4
finished parsing batch:  3 in :  3 fails:  0
finished parsing batch:  0 in :  22 fails:  0
finished parsing batch: in :  2  22 fails:  0
finished parsing batch:  1 in :  22 fails:  0
parse sqls took:  23
Num Ops:  79652 Num Expressions:  24605


In [380]:
if len(qreps) != 0:
#     for jid in set(exprdf["jobid"]):
#         qrep = qreps[int(jid)]
#         sql = sqls[int(jid)]
#         assert qrep["sql"].lower() == sql.lower()
#         tmp = exprdf[exprdf.jobid == jid]

    rowcounts = []
    totals = []
    sels = []
    
    for _,row in exprdf.iterrows():
        jid = row["jobid"]
        qrep = qreps[int(jid)]
        sql = sqls[int(jid)]
        try:
            cards = qrep["subset_graph"].nodes()[tuple([row["alias"],])]["cardinality"]
            rowcounts.append(cards["actual"])
            totals.append(cards["total"])
            sels.append(min(float(cards["actual"]) / cards["total"], 1.0))
        except:
            rowcounts.append(-1)
            totals.append(-1)
            sels.append(-1)

In [381]:
if len(qreps) == 0:
    exprdf["InputCardinality"] = -1
    exprdf["RowCount"] = -1
    exprdf["Selectivity"] = -1
else:
    exprdf["InputCardinality"] = totals
    exprdf["Selectivity"] = sels
    exprdf["RowCount"] = rowcounts

In [382]:
exprdf["Selectivity"].describe()

count    24605.000000
mean         0.115482
std          0.561440
min         -1.000000
25%          0.008850
50%          0.083333
75%          0.428571
max          1.000000
Name: Selectivity, dtype: float64

In [383]:
exprdf["filtersql"].values

array(["SELECT COUNT(*) from it AS it1 WHERE it1.id = '3'",
       'SELECT COUNT(*) from t AS t WHERE t.production_year <= 1975 AND 1875 < t.production_year',
       "SELECT COUNT(*) from mi AS mi2 WHERE mi2.info in ('Dutch','English','Finnish','German','Korean','Spanish','Tagalog')",
       ...,
       "SELECT COUNT(*) from rt AS rt WHERE rt.role in ('actor','actress')",
       "SELECT COUNT(*) from mi AS mi1 WHERE mi1.info in ('90')",
       'SELECT COUNT(*) from mii AS mii2 WHERE mii2.info::float <= 11.0 AND 7.0 <= mii2.info::float'],
      dtype=object)

In [384]:
if "stack" in WK:
    opdf["opstring"] = -1

In [385]:
exprdf.head(5)

,filtersql,jobid,input,alias,exprhash,unique_filter_cols,discrete_ops,cont_ops,like_ops,is_id,num_unique_ops,num_ops,num_discrete_consts,num_likes,like_lens,InputCardinality,Selectivity,RowCount
0,SELECT COUNT(*) from it AS it1 WHERE it1.id = '3',0,it,it1,201247606223691005679595717410545984913115377236,1,1,0,0,0,1,1,1,0,0,113,0.008850,1
1,SELECT COUNT(*) from t AS t WHERE t.production...,0,t,t,1352542446783672726478099788649831742465084335914,1,0,1,0,0,2,2,0,0,0,1053501,0.428637,451569
2,SELECT COUNT(*) from mi AS mi2 WHERE mi2.info ...,0,mi,mi2,1445181067565615963714989244604800786123047056644,1,1,0,0,0,1,7,7,0,0,6166815,0.158619,978172
3,SELECT COUNT(*) from it AS it2 WHERE it2.id = '4',0,it,it2,550750175548628326687047941186800664069875408065,1,1,0,0,0,1,1,1,0,0,113,0.008850,1
4,SELECT COUNT(*) from n AS n WHERE n.gender in ...,0,n,n,1412706111350165054529954383171903240337882399016,1,1,0,0,0,1,1,1,0,0,1739579,1.000000,1739579


In [386]:
print(opdf["is_id"].mean())
print(exprdf["is_id"].mean())

0.0
0.0


In [387]:
set(opdf["column"])

{'country_code',
 'gender',
 'id',
 'info',
 'info::float',
 'keyword',
 'kind',
 'name',
 'name_pcode_cf',
 'name_pcode_nf',
 'note',
 'production_year',
 'role',
 'surname_pcode',
 'title'}

In [388]:
if "stack" in WK:
    opdf = opdf[~opdf["input"].isin(["X", "function"])] 
    opdf["inplen"] = opdf.apply(lambda x: len(x["input"]) ,axis=1)
    opdf = opdf[opdf["inplen"] >= 4]
   
    exprdf = exprdf[~exprdf["input"].isin(["X", "function"])]
    exprdf["inplen"] = exprdf.apply(lambda x: len(x["input"]) ,axis=1)
    exprdf = exprdf[exprdf["inplen"] >= 4]

In [389]:
print("a" in set(opdf["input"]))

False


In [390]:
opdf["workload"] = WK
exprdf["workload"] = WK
opdf["db"] = WK
exprdf["db"] = WK

import time
start = time.time()

opfn = os.path.join(OUTPATH, "op_df.csv")
exprfn = os.path.join(OUTPATH, "expr_df.csv")

opdf.to_csv(opfn, index=False)
print("opdf to csv took: ", time.time()-start)

exprdf.to_csv(exprfn, index=False)

opdf to csv took:  0.7543506622314453


In [391]:
# opdf = opdf[opdf.input != "function"]
# opdf = opdf[opdf.input != "X"]
# exprdf = exprdf[exprdf.input != "function"]
# exprdf = exprdf[exprdf.input != "X"]
# print(len(opdf), len(exprdf))

In [356]:
exprdf.keys()

Index(['filtersql', 'jobid', 'input', 'alias', 'exprhash',
       'unique_filter_cols', 'discrete_ops', 'cont_ops', 'like_ops', 'is_id',
       'num_unique_ops', 'num_ops', 'num_discrete_consts', 'num_likes',
       'like_lens', 'InputCardinality', 'RowCount', 'Selectivity', 'workload',
       'db'],
      dtype='object')

In [357]:
opdf

,op,dtype,optype,constant,input,column,opstring,is_id,jobid,discrete_ops,cont_ops,like_ops,alias,exprhash,workload,db
0,=,string,=,'[us]',cn,country_code,cn.country_code = '[us]',0,0,1,0,0,cn,246246936916533429215669744677299380179211065442,job,job
1,=,string,=,'YouTube',cn,name,cn.name = 'YouTube',0,0,1,0,0,cn,246246936916533429215669744677299380179211065442,job,job
2,=,date,=,'release dates',it,info,it1.info = 'release dates',0,0,1,0,0,it1,201247606223691005679595717410545984913115377236,job,job
3,LIKE,num,3%,'%(200%)%',mc,note,mc.note like '%(200%)%',0,0,0,0,1,mc,1410753861686468872758163449862451826864997888115,job,job
4,LIKE,unknown,contains,'%(worldwide)%',mc,note,mc.note like '%(worldwide)%',0,0,0,0,1,mc,1410753861686468872758163449862451826864997888115,job,job
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,=,string,=,'rating',it,info,it2.info = 'rating',0,112,1,0,0,it2,1044850558769340704885184429602467397350107464301,job,job
1125,=,string,in,'Horror',mi,info,"mi.info in ('Horror',\n 'Thri...",0,112,1,0,0,mi,1425159986380809554144217888527206042206256450429,job,job
1126,=,string,in,'Thriller',mi,info,"mi.info in ('Horror',\n 'Thri...",0,112,1,0,0,mi,1425159986380809554144217888527206042206256450429,job,job
1127,>,cont,cont,'8.0',mi_idx,info,mi_idx.info > '8.0',0,112,0,1,0,mi_idx,788681035961215256139748819327589269367661515117,job,job


In [358]:
tmp = opdf[opdf.input == "date_dim"]
tmp = tmp[tmp.column == "d_year"]
tmp["opstring"].values

array([], dtype=object)

In [359]:
print(set(opdf["input"]))
tmp = opdf[opdf.input == "X"]
tmp

{'cct', 't', 'it', 'chn', 'mi', 'cn', 'mi_idx', 'ci', 'k', 'n', 'ct', 'kt', 'rt', 'an', 'lt', 'mc', 'pi'}


,op,dtype,optype,constant,input,column,opstring,is_id,jobid,discrete_ops,cont_ops,like_ops,alias,exprhash,workload,db


In [115]:
tmp = opdf[opdf.input == "function"]
tmp

,op,dtype,optype,constant,input,column,opstring,is_id,alias,jobid,discrete_ops,cont_ops,like_ops,exprhash,workload,db
6,<,cont,cont,0.9,function,function,"CAST(curr_yr.sales_cnt AS DECIMAL(17,2))/CAST(...",0,function,1,0,1,0,1194532054064400545969806492337960065944989809790,tpcds1,tpcds1
730,>,cont,cont,0,function,ws_qty,"coalesce(ws_qty,0)>0",0,function,50,0,1,0,352837534879430309755480276310447356833318897879,tpcds1,tpcds1
731,>,cont,cont,0,function,cs_qty,"coalesce(cs_qty, 0)>0",0,function,50,0,1,0,352837534879430309755480276310447356833318897879,tpcds1,tpcds1
871,>,cont,cont,0,function,function,(select count(*) as item_cnt\n from ite...,0,function,62,0,1,0,252541091547047920800273679976417847466813740638,tpcds1,tpcds1
879,>,cont,cont,1.2,function,function,(case when household_demographics.hd_vehicle_c...,0,function,64,0,1,0,481438817707749151144581095624954386821796428176,tpcds1,tpcds1


In [116]:
HTML(print_sql(sqls[59]))

NameError: name 'print_sql' is not defined

In [117]:
exprdf

,filtersql,jobid,input,alias,exprhash,unique_filter_cols,discrete_ops,cont_ops,like_ops,is_id,num_unique_ops,num_ops,num_discrete_consts,num_likes,like_lens,InputCardinality,RowCount,Selectivity,workload,db
0,SELECT COUNT(*) from item AS item WHERE item.i...,0,item,item,517422146408463513815778048196473408336958381784,1,1,0,0,1,1,1,1,0,0,-1,-1,-1,tpcds1,tpcds1
1,SELECT COUNT(*) from date_dim AS dt WHERE dt.d...,0,date_dim,dt,29594945970962246921075871724351446026691359122,2,1,0,0,0,1,2,2,0,0,-1,-1,-1,tpcds1,tpcds1
2,SELECT COUNT(*) from item AS item WHERE i_cate...,1,item,item,998969870485521328211605389720286787946524716786,1,1,0,0,0,1,1,1,0,0,-1,-1,-1,tpcds1,tpcds1
3,SELECT COUNT(*) from date_dim AS prev_yr WHERE...,1,date_dim,prev_yr,178771763793593387571164803369789816043736116950,1,1,0,0,0,1,1,1,0,0,-1,-1,-1,tpcds1,tpcds1
4,SELECT COUNT(*) from date_dim AS curr_yr WHERE...,1,date_dim,curr_yr,647632923169289817705511262092897855780512323282,1,1,0,0,0,1,1,1,0,0,-1,-1,-1,tpcds1,tpcds1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,SELECT COUNT(*) from store AS store WHERE stor...,101,store,store,184923325528354915412726819781338469089469494814,1,1,0,0,0,1,2,2,0,0,-1,-1,-1,tpcds1,tpcds1
258,SELECT COUNT(*) from date_dim AS date_dim WHER...,101,date_dim,date_dim,186590921360564454790375181920702142137017326247,2,1,1,0,0,3,5,3,0,0,-1,-1,-1,tpcds1,tpcds1
259,SELECT COUNT(*) from household_demographics AS...,101,household_demographics,household_demographics,880856604257465718987508440744238290871003793595,2,1,0,0,0,1,2,2,0,0,-1,-1,-1,tpcds1,tpcds1
260,SELECT COUNT(*) from date_dim AS d WHERE d1.d_...,102,date_dim,d,4777165936546303951770515401617716528552275365,1,1,0,0,0,1,7,7,0,0,-1,-1,-1,tpcds1,tpcds1


In [891]:
tmp = opdf[opdf.exprhash == 209065536981977045761885247808823910077020259304]
tmp

,op,dtype,optype,constant,input,column,opstring,alias,jobid,discrete_ops,cont_ops,like_ops,exprhash,workload,db


In [892]:
#exprdf.sort_values(by="unique_filter_cols", ascending=False).head(20)

In [893]:
exprdf[["unique_filter_cols", "num_likes"]].describe(percentiles=[0.9, 0.99])

,unique_filter_cols,num_likes
count,22.000000,22.000000
mean,2.181818,0.136364
std,1.531770,0.351250
min,1.000000,0.000000
50%,2.000000,0.000000
90%,3.900000,0.900000
99%,6.000000,1.000000
max,6.000000,1.000000


In [894]:
opdf.groupby("input")["column"].nunique().reset_index().sort_values(by="column", ascending=False).head(50)

,input,column
0,X,24
1,function,1
2,n,1


In [895]:
tmp = opdf[opdf.input == "posts"]
set(tmp["column"])

set()

In [624]:
tmp = exprdf[exprdf.unique_filter_cols == 3]
ehash = tmp["exprhash"].values[1]
tmp

IndexError: index 1 is out of bounds for axis 0 with size 0

In [419]:
tmp = opdf[opdf.exprhash == ehash]
tmp

,op,dtype,optype,constant,input,column,opstring,alias,qid,discrete_ops,cont_ops,like_ops,exprhash
476,=,num,=,3,date_dim,d_moy,=,date_dim,18,1,0,0,1449386839506961532079144159772542088486258583832
477,=,date,=,1999,date_dim,d_year,=,date_dim,18,1,0,0,1449386839506961532079144159772542088486258583832
478,between,cont,cont,-1,date_dim,d_month_seq,between,date_dim,18,0,0,0,1449386839506961532079144159772542088486258583832


In [413]:
def print_sql(sql):
    sql = sqlparse.format(sql, reindent=True, keyword_case='upper')
    sql = sql.replace("\n", "<br>")
    sql = sql.replace(" ", "&nbsp;")
    return sql

In [415]:
HTML(print_sql(sqls[93]))

In [186]:
#df = pd.concat(allpreds)

In [39]:
#df.to_csv("ceb_queries_df.csv", index=False)
#df.to_csv("tpcds_queries_df.csv", index=False)
#df.to_csv("stackexchange_queries_df.csv", index=False)

In [40]:
# df2 = pd.read_csv("job_queries_df.csv")
# df2.head(5)

In [312]:
HTML(opdf[["like_ops", "discrete_ops", "cont_ops"]].\
     describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))

index,like_ops,discrete_ops,cont_ops
count,1.099434e+06,1.099434e+06,1.099434e+06
mean,8.634170e-02,8.438624e-01,6.669068e-02
std,2.808681e-01,3.629859e-01,2.494856e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,1.000000e+00,0.000000e+00
90%,0.000000e+00,1.000000e+00,0.000000e+00
99%,1.000000e+00,1.000000e+00,1.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00


In [190]:
df = opdf

In [193]:
discdf = df[df.discrete_ops == 1]
#discdf.groupby(["dkind"]).count()["discrete_ops"].reset_index().sort_values(by="discrete_ops")

In [194]:
discdf.groupby(["dtype"]).count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

,dtype,discrete_ops
1,string,833
0,date,23


In [196]:
ldf = df[df.like_ops == 1]
ldf.groupby("dtype").count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

,dtype,discrete_ops
4,unknown,122
0,-1,32
3,short,23
1,num,13
2,serial,4


In [203]:
#ldf.groupby("likekind").count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

In [200]:
#inpdata = pd.DataFrame(allinpdata)

In [201]:
# print(len(inpdata))
# inpdata = inpdata.drop_duplicates()
# inpdata = inpdata[inpdata["col"] != "function"]
# print(len(inpdata))
# print(len(set(inpdata["col"])))
# print(set(inpdata["col"]))

# unique values seen in each column

In [202]:
#inpdata.groupby("col").count().reset_index()["val"].describe(percentiles=[0.75,0.9,0.99])

In [81]:
# dff = df[df["col"] == "function"]
# dff

,left,table,col,op,val,qid,dtype,dkind,likedtype,likekind,discrete_ops,in_ops,cont_ops,like_ops
3,(\n\t\t\tselect count(B.Id) \n\t\t\tfrom Posts...,function,function,>,0,66,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,N,X,function,>=,1,67,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,ParentId,X,function,=,135169,68,num,eq,-1,-1,1.0,0.0,0.0,0.0
0,Tags,X,function,like,'%google-cloud-functions%',71,-1,-1,serial,contains,0.0,0.0,0.0,1.0
0,PostTypeId,X,function,=,2,72,num,eq,-1,-1,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,(select sum(u3.Reputation) from Users u3 where...,function,function,>,1000,84765,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,PostHistoryTypeId,X,function,=,10,84766,num,eq,-1,-1,1.0,0.0,0.0,0.0
2,(up+down),function,function,>,0,84788,-1,-1,-1,-1,0.0,0.0,1.0,0.0
1,-2*down,function,function,<,0,84802,-1,-1,-1,-1,0.0,0.0,1.0,0.0


In [82]:
#pdf = pd.concat(allpreds)
pdf = df

In [83]:
pdf.head(5)

,left,table,col,op,val,qid,dtype,dkind,likedtype,likekind,discrete_ops,in_ops,cont_ops,like_ops
0,tcs.CusterStatus,tcs,CusterStatus,=,'Active',0,string,eq,-1,-1,1.0,0.0,0.0,0.0
0,posts.ViewCount,posts,ViewCount,>,10000,2,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,Posts.CreationDate,Posts,CreationDate,<,'2010-01-01',3,-1,-1,-1,-1,0.0,0.0,1.0,0.0
1,Posts.CreationDate,Posts,CreationDate,>,'2010-02-01',3,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,answer.PostTypeId,answer,PostTypeId,=,2,4,num,eq,-1,-1,1.0,0.0,0.0,0.0


In [84]:
#pdf = pdf[pdf.col.isin(COLS_TO_TABLES)]

# TODO: need equivalent version for SCOPE;
## TODO: more important is #columns per input in a query; for this we need column --> table mapping

In [85]:
# pdf.groupby(["col", "qid"])["op"].nunique().reset_index().sort_values(by="op", ascending=False)["op"].describe(
#                                     percentiles=[0.75,0.90,0.99])

count    76202.000000
mean         1.156964
std          0.421617
min          1.000000
50%          1.000000
75%          1.000000
90%          2.000000
99%          3.000000
max          6.000000
Name: op, dtype: float64

In [86]:
# for col in set(pdf["col"]):
#     if not col in COLS_TO_TABLES:
#         print(col)

nan

answer.Body
 reputation
m,CreationDate
DD,p.CreationDate
Users.LastAccessDate
ReviewTaskTypeId
m,q.creationdate
p.Id
owneruserid
description
c
AnswerCount
pp.Body
Rep
@anum
[Name]
dd,Question.CreationDate,Answer.CreationDate
ss,Users.CreationDate,Posts.CreationDate
ReviewResultType
[u].[Location]
V.CreationDate
P.Body
[Posttypeid]
q.Tags
Tags
[Status]
s,creationdate,closeddate
p.CommentCount
Monthindex
title
Answers.Body
q.Title
p.CreationDate,@latestDate
CommunityOwnedDate
b.Title
[Accepted]
b.Body
d,LastAccessDate
##TagName
p.Title
is_ms_shipped
Location
num
Users.DisplayName
[Order]
mi,p7.CreationDate,p6.CreationDate
VoteRevNo
Questions.CreationDate,Votes.CreationDate
phis.Comment
PostHistoryTypeId
us.DisplayName
v.CreationDate,p.CreationDate
yyyy,CreationDate
dd,users.CreationDate
index_id
dd,LastAccessDate
a.body
p.CreationDate,@EndDate
Posts.CreationDate,@latest
d,CreationDate
OwnerUserID
object_id
dd,CreationDate,LastPostedDate
CreationDate as char(10)
b.Date
MigrationHisto

In [87]:
# pdf2 = pdf[pdf.table != "X"]
# print(set(pdf2["table"]))
# print(len(set(pdf2["table"])))

{nan, 'vup', 'uc', 'pAll', 'csh', 'c', 'Results', 'mdi', 'subqAllAnswers', 'Ups', 'o1', 'qtn', 'CSH', 'me', 'v', 'A2', 'Tags', 'AnswerPosts', 'counts', 'TagCount', 'hp', 'TotalAnswers', '[#CustomerStatusHistory]', 'A', 'uu', 'num', 'PostHistory', '[ph]', 'postsq', 'ya', 'v2', 'AT', 'AcceptedAnswer', 'pall', 'e', 'ph0', 'u3', 'p1', 'qc', 'PostsA', 'tag1', 'pg', 'answer', 'a2', 'temp', 't', 'W', 'pa2', 'se', 'QUESTIONS', 'CloseSide', 'function', 'queries', 'f', 'wdv', 'uv', 'Bins', 'qbt', 'wk', 'accepted', 'actor', 'yourAnswer', 'AcceptVote', 'ht', 'y', 'ReviewTaskTypes', 'Y', 'naa', 'pt', 'bt', 'PA', 'k', 'answ', '[p]', 'tags', 'N', 'reply', 'fu', 'USERS', 'DS', 's1', 'vt', 'ContQ', 'Del', 'UserQuestions', 'Pht', 'hello', 'PF', 'ptypes', 'pans', 'prev', 'PythonBadges', 'votetypes', 'su', 'ReopenSide', 'ANSWERS', 'V', 'B', 'allTheTags', 'pc', 'Pt', 'comment', 'tagsY', 'pm', 'data0', 'this', 'v0', 'tagsX', 'status_tag', 'rollbacks', 'E', 'p_a', 'Raw', 'AcceptedAnswers', 'i2', '[se]', 'bty

In [88]:
# pdf2.groupby(["table", "qid"])["col"].nunique().reset_index().sort_values(by="col", ascending=False)["col"].describe(
#                                     percentiles=[0.75,0.90,0.99])

count    48325.000000
mean         1.207926
std          0.509182
min          1.000000
50%          1.000000
75%          1.000000
90%          2.000000
99%          3.000000
max          8.000000
Name: col, dtype: float64

In [89]:
# pd.set_option('display.max_rows', 500)
# pdf2.head(100)

,left,table,col,op,val,qid,dtype,dkind,likedtype,likekind,discrete_ops,in_ops,cont_ops,like_ops
0,tcs.CusterStatus,tcs,CusterStatus,=,'Active',0,string,eq,-1,-1,1.0,0.0,0.0,0.0
0,posts.ViewCount,posts,ViewCount,>,10000,2,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,Posts.CreationDate,Posts,CreationDate,<,'2010-01-01',3,-1,-1,-1,-1,0.0,0.0,1.0,0.0
1,Posts.CreationDate,Posts,CreationDate,>,'2010-02-01',3,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,answer.PostTypeId,answer,PostTypeId,=,2,4,num,eq,-1,-1,1.0,0.0,0.0,0.0
1,question.Score,question,Score,<,25,4,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,Votes.VoteTypeId,Votes,VoteTypeId,=,9,6,num,eq,-1,-1,1.0,0.0,0.0,0.0
0,Tags.TagName,Tags,TagName,=,'##Tag##',7,string,eq,-1,-1,1.0,0.0,0.0,0.0
4,A.CreationDate,A,OwnerUserId,>,'2016-01-01',7,num,in,-1,-1,1.0,1.0,0.0,0.0
5,A.OwnerUserId,NaN,NaN,in,"(\n 209103, -- puf\n 394010, -- katowulf...",7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# pdf2.groupby(["table", "qid"])["col"].nunique().reset_index().sort_values(by="col", ascending=False)

,table,qid,col
23073,function,30456,8
40790,q,57705,6
40198,q,32748,6
41359,q,81875,6
39920,q,21505,6
...,...,...,...
18007,d,25890,1
18008,d,26714,1
18009,d,30783,1
18011,d,39438,1


# TODO:

#### More columns being filtered (tails) on BUT within a column ---- fewer categorical values being hit
#### discrete_type: num vs string
#### cont dtypes: range vs leq vs geq

In [91]:
# pdf.groupby("op").count()["qid"]

op
!=            568
<            7646
<=           2780
<>           1214
=           93323
>           14509
>=           6438
between      1218
in           9267
like        41662
not in        517
not like     4972
Name: qid, dtype: int64